In [1]:
import pandas as pd
import numpy as np

In [2]:
from hashlib import md5
from sklearn.preprocessing import LabelEncoder

In [3]:
df = pd.read_csv("CiC_IoT23_merged.csv", low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46686579 entries, 0 to 46686578
Data columns (total 47 columns):
 #   Column           Dtype  
---  ------           -----  
 0   flow_duration    float64
 1   Header_Length    float64
 2   Protocol Type    float64
 3   Duration         float64
 4   Rate             float64
 5   Srate            float64
 6   Drate            float64
 7   fin_flag_number  float64
 8   syn_flag_number  float64
 9   rst_flag_number  float64
 10  psh_flag_number  float64
 11  ack_flag_number  float64
 12  ece_flag_number  float64
 13  cwr_flag_number  float64
 14  ack_count        float64
 15  syn_count        float64
 16  fin_count        float64
 17  urg_count        float64
 18  rst_count        float64
 19  HTTP             float64
 20  HTTPS            float64
 21  DNS              float64
 22  Telnet           float64
 23  SMTP             float64
 24  SSH              float64
 25  IRC              float64
 26  TCP              float64
 27  UDP       

In [4]:
# Create a hash for each column
def hash_column(series):
    return md5(pd.util.hash_pandas_object(series, index=False).values).hexdigest()

# Find columns with identical hashes
def find_identical_columns_by_hash(df):
    hash_dict = {}
    for col in df.columns:
        col_hash = hash_column(df[col])
        if col_hash in hash_dict:
            hash_dict[col_hash].append(col)
        else:
            hash_dict[col_hash] = [col]

    return [cols for cols in hash_dict.values() if len(cols) > 1]

# Applying the function to the DataFrame
identical_column_groups = find_identical_columns_by_hash(df)
print("Groups of identical columns:", identical_column_groups)

Groups of identical columns: [['Rate', 'Srate'], ['IPv', 'LLC']]


In [5]:
# Groups of identical columns
identical_column_groups = [
    ['Rate', 'Srate'],
    ['IPv', 'LLC']
]

# Iterate through the list of groups and drop all but the first column
for group in identical_column_groups:
    # Keep the first column of the group and drop the rest
    columns_to_drop = group[1:]  # all columns except the first one
    df = df.drop(columns_to_drop, axis=1)

In [6]:
df.shape

(46686579, 45)

In [7]:
dtypes = {
        'flow_duration': np.float32,
        'Header_Length': np.uint32,
        'Protocol Type': np.uint16,
        'Duration': np.float32,
        'Rate': np.uint32,
        'Drate': np.float32,
        'fin_flag_number': np.bool_,
        'syn_flag_number': np.bool_,
        'rst_flag_number': np.bool_,
        'psh_flag_number': np.bool_,
        'ack_flag_number': np.bool_,
        'ece_flag_number': np.bool_,
        'cwr_flag_number': np.bool_,
        'ack_count': np.float16,
        'syn_count': np.float16,
        'fin_count': np.uint16,
        'urg_count': np.uint16, 
        'rst_count': np.uint16, 
        'HTTP': np.bool_, 
        'HTTPS': np.bool_, 
        'DNS': np.bool_, 
        'Telnet': np.bool_,
        'SMTP': np.bool_, 
        'SSH': np.bool_, 
        'IRC': np.bool_, 
        'TCP': np.bool_, 
        'UDP': np.bool_, 
        'DHCP': np.bool_, 
        'ARP': np.bool_, 
        'ICMP': np.bool_, 
        'IPv': np.bool_, 
        'Tot sum': np.float32, 
        'Min': np.float32, 
        'Max': np.float32, 
        'AVG': np.float32, 
        'Std': np.float32, 
        'Tot size': np.float32, 
        'IAT': np.float32, 
        'Number': np.float32,
        'Magnitue': np.float32, 
        'Radius': np.float32, 
        'Covariance': np.float32, 
        'Variance': np.float32, 
        'Weight': np.float32, 
        'label': str
    }

def convert_dtype(df):
    # Adjust data type
    for col,typ in dtypes.items():
        df[col] = df[col].astype(typ)   
    
    
    # Fix spelling error in original dataset
    df['Magnitude'] = df['Magnitue']
    return df.drop(['Magnitue'], axis=1)

df = convert_dtype(df)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46686579 entries, 0 to 46686578
Data columns (total 45 columns):
 #   Column           Dtype  
---  ------           -----  
 0   flow_duration    float32
 1   Header_Length    uint32 
 2   Protocol Type    uint16 
 3   Duration         float32
 4   Rate             uint32 
 5   Drate            float32
 6   fin_flag_number  bool   
 7   syn_flag_number  bool   
 8   rst_flag_number  bool   
 9   psh_flag_number  bool   
 10  ack_flag_number  bool   
 11  ece_flag_number  bool   
 12  cwr_flag_number  bool   
 13  ack_count        float16
 14  syn_count        float16
 15  fin_count        uint16 
 16  urg_count        uint16 
 17  rst_count        uint16 
 18  HTTP             bool   
 19  HTTPS            bool   
 20  DNS              bool   
 21  Telnet           bool   
 22  SMTP             bool   
 23  SSH              bool   
 24  IRC              bool   
 25  TCP              bool   
 26  UDP              bool   
 27  DHCP      

In [ ]:
df = df.drop_duplicates()
df.shape

In [ ]:
# Log-transform skewed columns
num_cols = df.select_dtypes(include=['number']).columns

skewness = df[num_cols].skew()

cols_to_log = skewness[(skewness > 10) | (skewness.isna())].index.tolist()

print(f"Columnas a transformar con log1p ({len(cols_to_log)}):")
print(cols_to_log)

df[cols_to_log] = np.log1p(df[cols_to_log])

In [ ]:
#Save to .parquet
df.to_parquet('CiC_IoT23_clean.parquet')